In [1]:
# imports
from laplace import Laplace

import numpy as np
import pandas as pd
import torch

import json

import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

torch.manual_seed(43)

In [2]:
# Import data from csv

# Load data from CSV file using pandas
df = pd.read_csv('data_regression.csv')

# Split the dataframe into x and y tensors
data = torch.from_numpy(df[['xs', 'y']].to_numpy()).to(torch.float32)
x = torch.from_numpy(df['xs'].to_numpy()).to(torch.float32)
y = torch.from_numpy(df['y'].to_numpy()).to(torch.float32)

#y_unique = torch.unique(y)
#y_indices = y - 1
#y_indices = y_indices.long()
#y_train = nn.functional.one_hot(y_indices, num_classes=len(y_unique)).float()
#y_train

In [3]:
# Init model

n_hidden = 10
D = 1
out_dim = 1 
model = nn.Sequential(
    nn.Linear(D, n_hidden),
    nn.Sigmoid(),
    nn.Linear(n_hidden, out_dim)
)

loss_fn = nn.CrossEntropyLoss()

In [4]:
with open('nn_regression.json') as fin:
    nn_json_str = fin.read()
    nn_json = json.loads(nn_json_str)

nn_json

[{'bias': [-1.1201183,
   -1.0552441,
   -1.1727674,
   1.0405303,
   1.1344997,
   -1.0677602,
   1.9962846,
   1.04113,
   -2.8023236,
   1.1533123],
  'weight': [[0.25581408,
    0.24827705,
    0.25862476,
    0.4965463,
    -1.3918695,
    0.24902664,
    -0.7361471,
    -0.24590988,
    0.7512668,
    -0.6878244]]},
 {'bias': [0.5303483],
  'weight': [[1.339898],
   [0.9142657],
   [0.85277814],
   [0.45185313],
   [-0.4072362],
   [0.86708283],
   [1.2918717],
   [-0.77205473],
   [-1.8449564],
   [0.4374265]]}]

In [5]:
assert len(model.state_dict()) == 2 * len(nn_json)
iter_states = iter(model.state_dict())

# for layer in model.state_dict():
#     print(layer)
for layer_read in nn_json:
    state_w = next(iter_states)
    state_b = next(iter_states)
    tensor_w = torch.tensor(layer_read['weight']).T
    tensor_b = torch.tensor(layer_read['bias']).T
    model.state_dict()[state_w].data.copy_(tensor_w)
    model.state_dict()[state_b].data.copy_(tensor_b)


C:\Users\adeli\AppData\Local\Temp\ipykernel_29104\3249112778.py:10: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3281.)
  tensor_b = torch.tensor(layer_read['bias']).T


In [6]:
for val in model.state_dict().values():
    display(val)

tensor([[ 0.2558],
        [ 0.2483],
        [ 0.2586],
        [ 0.4965],
        [-1.3919],
        [ 0.2490],
        [-0.7361],
        [-0.2459],
        [ 0.7513],
        [-0.6878]])

tensor([-1.1201, -1.0552, -1.1728,  1.0405,  1.1345, -1.0678,  1.9963,  1.0411,
        -2.8023,  1.1533])

tensor([[ 1.3399,  0.9143,  0.8528,  0.4519, -0.4072,  0.8671,  1.2919, -0.7721,
         -1.8450,  0.4374]])

tensor([0.5303])

In [7]:
la = Laplace(model, 'regression',
             subset_of_weights='all',
             hessian_structure='full')

X = x.unsqueeze(1)

la.fit(DataLoader(TensorDataset(X, y), batch_size=1))

hessian = la.H
torch.set_printoptions(precision=4, sci_mode=True)
print(hessian)

c:\Users\adeli\OneDrive\Desktop\facultate\2nd year\Q4 - Software Project\LaplaceRedux.jl\myenv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor([[ 6.1229e+02,  4.1711e+02,  3.9173e+02,  2.1577e+01, -6.8919e+00,
          3.9609e+02,  2.4293e+02, -3.5245e+02, -4.8363e+02,  5.7804e+01,
          1.1815e+02,  8.0638e+01,  7.5375e+01,  5.9110e+00, -3.3075e+00,
          7.6526e+01,  5.8729e+01, -6.8142e+01, -1.0526e+02,  1.5174e+01,
          2.0757e+02,  2.0985e+02,  2.0421e+02,  3.6163e+02,  1.1458e+01,
          2.0908e+02,  7.5008e+01,  1.6580e+02,  2.6270e+02,  4.8867e+01,
          3.7581e+02],
        [ 4.1711e+02,  2.8416e+02,  2.6685e+02,  1.4749e+01, -4.7497e+00,
          2.6984e+02,  1.6586e+02, -2.4010e+02, -3.2980e+02,  3.9503e+01,
          8.0638e+01,  5.5036e+01,  5.1440e+01,  4.0581e+00, -2.2897e+00,
          5.2229e+01,  4.0195e+01, -4.6508e+01, -7.1905e+01,  1.0403e+01,
          1.4151e+02,  1.4307e+02,  1.3921e+02,  2.4676e+02,  7.9509e+00,
          1.4255e+02,  5.1509e+01,  1.1331e+02,  1.7893e+02,  3.3587e+01,
          2.5650e+02],
        [ 3.9173e+02,  2.6685e+02,  2.5063e+02,  1.3727e+01, -4.34